# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [3]:
import pandas as pd

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [62]:
data = pd.read_csv('data/labeled.csv')

In [220]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [63]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

### Default

In [64]:
vectorizer_def = TfidfVectorizer(min_df=5, max_df=0.4)
X = vectorizer_def.fit_transform(train.comment)
X_test = vectorizer_def.transform(test.comment) 

In [65]:
y = train.toxic.values
y_test = test.toxic.values

In [66]:
clf = LogisticRegression(C=0.1, class_weight='balanced')

In [67]:
clf.fit(X, y)
preds = clf.predict(X_test)

In [68]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.83      0.85       953
         1.0       0.70      0.78      0.73       489

    accuracy                           0.81      1442
   macro avg       0.79      0.80      0.79      1442
weighted avg       0.82      0.81      0.81      1442



### Razdel

In [42]:
import razdel

In [50]:
def tokenize_with_razdel(text):
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    tokens = [[token.text for token in razdel.tokenize(sentence)] for sentence in sentences]
    ts = []
    for sent in tokens:
        for word in sent:
            ts.append(word)
    return ts

In [69]:
vectorizer_raz = TfidfVectorizer(min_df=5, max_df=0.4, tokenizer=tokenize_with_razdel)
X1 = vectorizer_raz.fit_transform(train.comment)
X1_test = vectorizer_raz.transform(test.comment) 

c:\Users\fenlich\miniconda3\envs\barbiturat\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [70]:
y1 = train.toxic.values
y1_test = test.toxic.values

In [71]:
clf1 = LogisticRegression(C=0.1, class_weight='balanced')
clf1.fit(X1, y1)
preds1 = clf1.predict(X1_test)

In [72]:
print(classification_report(y1_test, preds1))

              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85       953
         1.0       0.69      0.77      0.73       489

    accuracy                           0.81      1442
   macro avg       0.78      0.80      0.79      1442
weighted avg       0.81      0.81      0.81      1442



### Вывод
Результат с кастомной токенизацией чуть-чуть лучше (precision, recall, avgs), accuracy одинаковая

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [74]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [367]:
from nltk.corpus import stopwords
russian_stopwords = list(stopwords.words('russian'))

### MultinomialNB

In [ ]:
vectorizer_NB = CountVectorizer(min_df=5, max_features=10000, max_df=0.4, stop_words=russian_stopwords, ngram_range=(1, 2))
X_NB = vectorizer_NB.fit_transform(train.comment)
X_NB_test = vectorizer_NB.transform(test.comment)
y_NB = train.toxic.values
y_NB_test = test.toxic.values

In [ ]:
clf_NB = MultinomialNB(alpha=1, force_alpha=True)
clf_NB.fit(X_NB, y_NB)
predsNB = clf_NB.predict(X_NB_test)

print(classification_report(y_NB_test, predsNB, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89       960
         1.0       0.81      0.70      0.75       482

    accuracy                           0.85      1442
   macro avg       0.84      0.81      0.82      1442
weighted avg       0.84      0.85      0.84      1442



In [276]:
res1 = clf_NB.predict_log_proba(X_NB_test)
top1 = res1[:, 1].argsort()[-10:]
test.loc[top1, 'comment'].values

array(['Злобный дегенерат и педераст, здохший от передоза в говне и блевотине, от которого даже отец отрекся, вновь возвращается, чтобы доводить детей до анальных колик. Суть такова, что на телефон поступает видеодозвон, где кровожадный педофил, видать только слезший с кукана Сатаны, угрожает придти к вам в спальню в три часа ночи и сделать с вами ЭТО под незабвенные истерические взгвизгивания йи-хи ! Вся Северная Америка в панике! Оперативные сводки доносят, что форс начался в Мексике и распространился на весь континент! Общественность трясется от ужаса и негодует! Невзирая на заверения полиции, что это всего лишь безобидный форс и никто не собирается насиловать вашу спидозную жопу, граждане не верят и вооружаются на случай всякого, намереваясь встретить инфернального педофила хэдшотом в ебало. Так же распространяются сертифицированные памятки об анальной угрозе со стороны сдохшего баребаки. Так что будьте бдительны, может быть в это самое время Майкл Джексон угрожает изнасиловать и в

### LogisticRegression

In [ ]:
vectorizer_LR = TfidfVectorizer(min_df=5, max_features=11000, max_df=0.5, stop_words=russian_stopwords, ngram_range=(1, 2))
X_LR = vectorizer_LR.fit_transform(train.comment)
X_LR_test = vectorizer_LR.transform(test.comment)
y_LR = train.toxic.values
y_LR_test = test.toxic.values

In [219]:
clf_LR = LogisticRegression(C=0.9, class_weight='balanced')
clf_LR.fit(X_LR, y_LR)
predsLR = clf_LR.predict(X_LR_test)

print(classification_report(y_LR_test, predsLR))

              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       960
         1.0       0.73      0.77      0.75       482

    accuracy                           0.83      1442
   macro avg       0.80      0.81      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [ ]:
res2 = clf_NB.predict_log_proba(X_LR_test)
top2 = res2[:, 1].argsort()[-10:]
a = test.loc[top2, 'comment'].values

array(['null 0 Сука, какие же коммибляди тупые.\n',
       'Э че бля а? Ты че нах бл иди сюда епты.. Ты за царя гнида? Я тя щас грязь растопчу хуесос ептыбля\n',
       'Тебе Егор, блядь, все мозги засрал, идиот? Путин устроил войну, выгодную только ему,блядь, они её поддержали(действия Путина), что ты втираешь мне? Свой народ?! Ахуенный народ, блядь - орда новиопов всех мастей со всей России, а не местных, у меня сосед по дому туда ездил - он тот ещё аутист и полудурок, а не русский националист. Вы изгадили Русский национализм, кретины!\n',
       'У хохлов теперь томос есть. То есть нихуя они не провославные, а сектанты ебанные и мрази. А у русни его нет и никогда не было. Учи матчасть, чтобы не быть батхёртом\n',
       'Дык малолетний долбоёб поди и правда дебил как и его мамаша-шлюха.\n',
       'Сириусли? Ну мразь ебаная, в край уже охуела и хули вы там сидите-пердите, надо этой шкуре показать, шо ее место под шконкой. Мразь охуевшая, а по ебасосине ее тупорылой не задонатить? Пи

### Вывод
Тексты действительно токсичные, часть из них совпадают (3 совпадения)

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

### Logistic Regression

In [368]:
vectorizer = TfidfVectorizer(min_df=5, max_features=11000, max_df=0.5, stop_words=russian_stopwords, ngram_range=(1, 2))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

In [369]:
features = vectorizer.get_feature_names_out()

In [370]:
reg = LogisticRegression(C=0.9, class_weight='balanced')
reg.fit(X, y)
preds = reg.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       960
         1.0       0.73      0.77      0.75       482

    accuracy                           0.83      1442
   macro avg       0.80      0.81      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [285]:
import numpy as np

In [371]:
importance = np.abs(reg.coef_[0]) #тут модуль
top = importance.argsort()[-5:][::-1] #получаем топ 5 индексов
result =[features[i] for i in top] #сопоставляем признаки с их индексами
result

['хохлы', 'хохлов', 'тебе', 'спасибо', 'нахуй']

### Decision Tree

In [372]:
vectorizer = TfidfVectorizer(min_df=5, max_features=1000, max_df=0.5, stop_words=russian_stopwords, ngram_range=(1, 2))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values
features = vectorizer.get_feature_names_out()

In [375]:
dectree = DecisionTreeClassifier(max_depth=2000, min_samples_split=5, class_weight='balanced')
dectree.fit(X, y)
preds = dectree.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.82      0.76      0.79       960
         1.0       0.58      0.67      0.62       482

    accuracy                           0.73      1442
   macro avg       0.70      0.71      0.71      1442
weighted avg       0.74      0.73      0.73      1442



In [374]:
importance = dectree.feature_importances_
top = importance.argsort()[-5:][::-1]
result =[features[i] for i in top]
result

['это', 'очень', 'тебе', 'хохлы', 'нахуй']

### Random Forest

In [343]:
vectorizer = TfidfVectorizer(min_df=5, max_features=10000, max_df=0.4, stop_words=russian_stopwords, ngram_range=(1, 2))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values
features = vectorizer.get_feature_names_out()

In [347]:
rf = RandomForestClassifier(max_depth=1000, n_estimators=120, min_samples_split=4)
rf.fit(X, y)
preds = rf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.80      0.93      0.86       960
         1.0       0.79      0.53      0.64       482

    accuracy                           0.80      1442
   macro avg       0.80      0.73      0.75      1442
weighted avg       0.80      0.80      0.78      1442



In [348]:
importance = rf.feature_importances_
top = importance.argsort()[-5:][::-1]
result =[features[i] for i in top]
result

['это', 'хохлы', 'хохлов', 'тебе', 'нахуй']

### Naive Bayes

In [349]:
vectorizer = CountVectorizer(min_df=5, max_features=10000, max_df=0.4, stop_words=russian_stopwords, ngram_range=(1, 2))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values
features = vectorizer.get_feature_names_out()

In [350]:
bayes = MultinomialNB(alpha=1, force_alpha=True)
bayes.fit(X, y)
preds = bayes.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89       960
         1.0       0.81      0.70      0.75       482

    accuracy                           0.85      1442
   macro avg       0.84      0.81      0.82      1442
weighted avg       0.84      0.85      0.84      1442



In [352]:
importance = np.abs(bayes.feature_log_prob_[0])
top = importance.argsort()[-5:][::-1]
result =[features[i] for i in top]
result

['пидораха', 'ватники', 'поссал', 'бандитов', 'пошел нахуй']